In [39]:
from io import StringIO
from scipy.io import mmread
import scipy
import numpy as np
import math
import timeit
def jacobi(A, b, tolerance, max_iter, x):
    n = len(x)
    x_old = x.copy()  #Set x_old intitally
    prev_diff = np.inf
    for k in range(max_iter):
        x_new = np.zeros_like(x)  # New empty x
        
        for j in range(n):
            # Sum of values excluding diagonal
            sum = np.dot(A[j, :], x_old) - A[j, j] * x_old[j]
            
            # Update x
            x_new[j] = (b[j] - sum) / A[j, j]
        
        # Check for convergence
        diff = np.max(np.abs(x_new - x_old)) / np.max(np.abs(x_old))

        #Was getting divergence so this is essentally to check if it goes to inf or nan
        if diff >= prev_diff+100000:
            print(f"Method Diverged at iteration {k+1}.")
            break
        prev_diff = diff
        # update values
        x_old = x_new.copy()
        
        if diff < tolerance:
            break
    return x_new
def gauss_seidel(A, b, tolerance, max_iter, x):
    n = len(x)
    for k in range(max_iter):
        x_old = x.copy()
        for i in range(n):
            # updated x values
            sum1 = np.dot(A[i, :i], x[:i])
            
           #old values sum
            sum2 = np.dot(A[i, i+1:], x_old[i+1:])
            #compute new x
            x[i] = (b[i] - sum1 - sum2) / A[i, i]
        
        # Test if method converges
        diff = np.max(np.abs(x - x_old)) / np.max(np.abs(x_old))
        if diff < tolerance:
            break
    return x    
            



def generate_diagonally_dominant_matrix(n):
    #random matrix
    A = np.random.rand(n, n)
    for i in range(n):
        row_sum = np.sum(np.abs(A[i, :])) - np.abs(A[i, i])  
        A[i, i] = row_sum + np.random.rand() + 1 #make diagonal element bigger
    
    return A

def check_solution_with_tolerance(A, x, b, tolerance=.1):
    # Calculate A * x
    b_computed = np.dot(A, x)
    
    # Check to see if x is a correct solution
    if np.allclose(b_computed, b, atol=tolerance):
        print("X is a correct solution based on the tolerance")
    else:
        print("The outputted b does not match the original b")

def generate_complex_symmetric_indefinite_matrix(n):
    # Step 1: Generate a random complex matrix
    A_real = np.random.randn(n, n)  # real part
    A_imag = np.random.randn(n, n)  # imaginary part
    A = A_real + 1j * A_imag

    # Step 2: Make the matrix symmetric
    A = (A + A.T) / 2

    # Step 3: Modify diagonal elements to ensure diagonal dominance
    for i in range(n):
        A[i, i] = np.sum(np.abs(A[i])) + 1  # ensure diagonal dominance

    # Step 4: Make the matrix indefinite by flipping the sign of some diagonal elements
    num_negative_diag = n // 2  # roughly half negative diagonal entries
    for i in range(num_negative_diag):
        A[i, i] = -np.abs(A[i, i])

    return A

#Reading in Matrix
A = mmread("bcsstm21.mtx").toarray()

"""I could not get the methods to converge on the given matrix. I suspect it is because the matrix listed is not diagonally dominant,
 I think this is leading to divergence so I generated this little script to get a diagonally dominant matrix and everything converged"""
A = generate_diagonally_dominant_matrix(180)
#Params
tolerance = .0001
max_iter = 50000
x_true = np.random.rand(180)
X0 = np.random.rand(180)
#b = np.dot(A, x_true)
#Methods
#x1 = jacobi(A,b,tolerance,max_iter,X0)
#x2 = gauss_seidel(A,b,tolerance,max_iter,X0)
#Check for convergence
L = scipy.linalg.cholesky(A, lower=False, overwrite_a=False, check_finite=True)
#print(L)

def time_jacobi():
    X0 = np.random.rand(180)
    b = np.dot(A, x_true)
    jacobi(A, b, tolerance, max_iter, X0)

jacobi_time = timeit.timeit(time_jacobi, number=100)
print(f"Average Jacobi execution time over 10 runs: {jacobi_time / 100} seconds")

# Time the Gauss-Seidel method
def time_gauss_seidel():
    X0 = np.random.rand(180)
    b = np.dot(A, x_true)
    gauss_seidel(A, b, tolerance, max_iter, X0)

gauss_seidel_time = timeit.timeit(time_gauss_seidel, number=100)
print(f"Average Gauss-Seidel execution time over 10 runs: {gauss_seidel_time / 100} seconds")

# Time the Cholesky decomposition
def time_cholesky():
    n = 180
    A = A = generate_diagonally_dominant_matrix(180)
    x_true = np.random.rand(n) + 1j * np.random.rand(n)
    b = np.dot(A, x_true)
    x0 = np.zeros(n, dtype=complex)
    L = scipy.linalg.cholesky(A, lower=True)
    y = scipy.linalg.solve_triangular(L, b, lower=True)
    x_cholesky = scipy.linalg.solve_triangular(L.conj().T, y, lower=False)
    
cholesky_time = timeit.timeit(time_cholesky, number=100)
print(f"Average Cholesky execution time over 10 runs: {cholesky_time / 100} seconds")
#L = Cholesky_Decomposition(A, 180)
#check_solution_with_tolerance(A,x1,b,.01)
#check_solution_with_tolerance(A,x2,b,.01)


Average Jacobi execution time over 10 runs: 0.06403367000049912 seconds
Average Gauss-Seidel execution time over 10 runs: 0.004223535000346601 seconds
Average Cholesky execution time over 10 runs: 0.2969783687510062 seconds
